In [1]:
# Setup Python import path
import sys
import os

# Adjust the path to your build directory where neurosim.so/.pyd exists
build_dir = os.path.expanduser('~/source/NeuroSim/build')
if build_dir not in sys.path:
    sys.path.insert(0, build_dir)

# Imports
import neurosim
import matplotlib.pyplot as plt
import numpy as np

# Enable inline plotting (only in Jupyter)
%matplotlib inline


In [2]:
def run_simulation():
    """Run a simple voltage trace demo using the NeuroSim Python module."""
    sim = neurosim.Simulation(10, 10)
    sim.set_input_current(10.0)
    sim.set_selected_neuron(5)

    time_steps = 200
    dt = 0.5  # ms
    voltage_trace = []

    print("Running simulation...")

    for _ in range(time_steps):
        sim.step(dt)
        voltage = sim.neurons()[sim.get_selected_neuron()].get_voltage()
        voltage_trace.append(voltage)

    # Plot membrane voltage trace
    plt.figure(figsize=(10, 4))
    plt.plot(np.arange(time_steps) * dt, voltage_trace, label='Voltage Trace')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (mV)')
    plt.title('Neuron Voltage Trace (Neuron {})'.format(sim.get_selected_neuron()))
    plt.grid(True)
    plt.legend()
    plt.show()

    # Heatmap visualization
    width, height = 9, 9
    grid = sim.get_voltage_grid(width, height)
    heatmap = np.array(grid).reshape((height, width))

    plt.figure(figsize=(6, 6))
    plt.imshow(heatmap, cmap='inferno', interpolation='nearest', vmin=0, vmax=1)
    plt.colorbar(label='Normalized Voltage')
    plt.title('Voltage Heatmap')
    plt.show()

run_simulation()

AttributeError: 'neurosim.Simulation' object has no attribute 'set_input_current'

In [ ]:
def run_simulation_with_synapse():
    """Run a demo where neuron 0 drives neuron 5 via a strong synapse."""
    sim = neurosim.Simulation(10)
    sim.set_input_current(5.0)
    sim.add_synapse(0, 5, 10.0)
    sim.set_selected_neuron(5)

    time_steps = 200
    dt = 0.5
    voltage_trace = []

    print("Running simulation with synapse...")

    for _ in range(time_steps):
        sim.step(dt)
        voltage = sim.neurons()[sim.get_selected_neuron()].get_voltage()
        voltage_trace.append(voltage)

    # Plot trace of post-synaptic neuron
    plt.figure(figsize=(10, 4))
    plt.plot(np.arange(time_steps) * dt, voltage_trace, label='Neuron 5 (Post)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (mV)')
    plt.title('Post-Synaptic Response Driven by Neuron 0')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Optional heatmap
    voltage_grid = sim.get_voltage_grid(3, 4)
    heatmap_data = np.array(voltage_grid).reshape((4, 3))

    plt.figure(figsize=(6, 6))
    plt.imshow(heatmap_data, cmap='viridis', interpolation='nearest', vmin=0, vmax=1)
    plt.colorbar(label='Normalized Voltage')
    plt.title('Voltage Heatmap')
    plt.show()

run_simulation_with_synapse()

In [ ]:
def plot_firing_rate_curve():
    currents = np.linspace(0, 20, 21)
    rates = []

    for current in currents:
        sim = neurosim.Simulation(1)
        sim.set_input_current(current)
        time_steps = 1000
        dt = 0.5
        for _ in range(time_steps):
            sim.step(dt)
        rate = sim.get_spike_rate(0, 500)
        rates.append(rate)

    plt.figure(figsize=(8, 4))
    plt.plot(currents, rates, marker='o')
    plt.xlabel("Input Current (nA)")
    plt.ylabel("Firing Rate (Hz)")
    plt.title("Neuron F-I Curve")
    plt.grid(True)
    plt.show()

plot_firing_rate_curve()

In [ ]:
def run_bursting_network():
    sim = neurosim.Simulation(30)
    sim.set_input_current(8.0)
    sim.connect_random(0.1, weight=5.0)

    steps = 500
    dt = 0.5
    spikes = []

    for _ in range(steps):
        sim.step(dt)
        for t, idx in sim.spike_events():
            spikes.append((t, idx))

    times, neurons = zip(*spikes)
    plt.figure(figsize=(10, 5))
    plt.scatter(times, neurons, s=5)
    plt.xlabel("Time (ms)")
    plt.ylabel("Neuron Index")
    plt.title("Raster Plot of Spiking Activity")
    plt.show()

run_bursting_network()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def interactive_voltage():
    sim = neurosim.Simulation(1)

    def update(current=5.0):
        sim.set_input_current(current)
        sim.set_selected_neuron(0)
        trace = []
        for _ in range(200):
            sim.step(0.5)
            v = sim.neurons()[0].get_voltage()
            trace.append(v)
        plt.figure(figsize=(8, 3))
        plt.plot(trace)
        plt.title(f"Membrane Voltage (I = {current} nA)")
        plt.grid(True)
        plt.show()

    slider = widgets.FloatSlider(min=0.0, max=20.0, step=0.5, value=5.0)
    widgets.interact(update, current=slider)


In [ ]:
def heterogeneous_population():
    sim = neurosim.Simulation(20)
    for i, n in enumerate(sim.neurons()):
        tau = 10 + i  # increasing time constant
        # Assume IntegrateAndFireNeuron constructor allows setting tau
        # n.set_tau(tau)  # hypothetical API

    sim.set_input_current(12.0)
    steps = 500
    dt = 0.5
    all_traces = [[] for _ in range(20)]

    for _ in range(steps):
        sim.step(dt)
        for i in range(20):
            all_traces[i].append(sim.neurons()[i].get_voltage())

    plt.figure(figsize=(10, 6))
    for i, trace in enumerate(all_traces):
        plt.plot(np.array(trace) + i * 100, label=f"Neuron {i}")
    plt.title("Membrane Voltages Across Diverse Neurons")
    plt.xlabel("Time Step")
    plt.ylabel("Voltage (Offset)")
    plt.show()
